In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from hmmlearn.hmm import CategoricalHMM
from sklearn.metrics import log_loss, roc_auc_score

In [ ]:
data = pd.read_csv("../Data/sample.csv")

In [ ]:
data = data.sort_values(by=['user_xid', 'skill_id', 'start_time'])

In [ ]:
grouped = data.groupby(by=['skill_id', 'user_xid'])
grouped.head()

In [ ]:
# f1 = open("group.txt", "w")

curr_skill = 1
skill_dict = {}
value_dict = {"seq": [], "lengths": []}

# count = 0
for value, group in tqdm(grouped):
    # if count == 100:
    #     break    
    # border = '===============================================\n'
    # group_string = str(group['discrete_score'])
    # write_value = border + 'skill_id: ' + str(value[0]) + '\n' + 'user_xid: ' + str(value[1]) + '\n' + group_string + '\n\n'
    # f1.write(write_value)
    
    if value[0] != curr_skill:
        value_dict['seq'] = np.concatenate(value_dict['seq']).reshape(-1,1)
        skill_dict[curr_skill] = value_dict
        value_dict = {"seq": [], "lengths": []}
    
    curr_skill = value[0]
    value_dict['seq'].append(group.discrete_score.to_numpy())
    value_dict['lengths'].append(group.shape[0])
    # count += 1
value_dict['seq'] = np.concatenate(value_dict['seq']).reshape(-1,1).astype(int)
skill_dict[curr_skill] = value_dict
    
# f1.close()
    

In [ ]:
start_prob = np.array([0.5, 0.5])
trans_prob = np.array([[0.7, 0.3],[0.3, 0.7]])
emission_prob = np.array([[0.8, 0.2],[0.2, 0.8]])

model = CategoricalHMM(n_components=2, init_params="", params="st", startprob_prior=start_prob, transmat_prior=trans_prob, emissionprob_prior=emission_prob, n_iter=100, verbose=True)

In [ ]:
count = 0
for skill in skill_dict:
    if count > 10:
        break    
        
    X = skill_dict[skill]['seq']
    lengths = skill_dict[skill]['lengths']
    
    model.fit(X, lengths)
    
    print("Finished model training. Printing final statistics...")

    # print("Finished model training. Printing final statistics...")
    # y_score = model.predict_proba(X)[:, 1]
    # 
    # loss = log_loss(X, y_score)     # logistical loss
    # auc = roc_auc_score(X, y_score)
    # 
    # print(f"Loss: {loss}")
    # print(f"AUC:  {auc}")
    
    count += 1